# Refatorando o dataset para facilitar a sua análise

## Imports

In [ ]:
import pandas as pd

import ast

from difflib import SequenceMatcher

## Funções

In [ ]:
def convert_k_to_int(linha):
    if 'k' in linha:
        linha = linha.replace("k", "")
        linha = float(linha) * 1000
    else:
        linha = float(linha)
    return linha

def remove_comma(linha):
    linha = str(linha).replace(",", "")
    return linha

## Refatorando

In [ ]:
# Dataset original
data_original = pd.read_csv('Github_data.txt')

# Dataset refatorado
data_refatorado = data_original.copy()

In [ ]:
data_original

In [ ]:
# Trocando o "k"(mil) para int(1000) na coluna "star"
data_refatorado['star'] = data_refatorado['star'].apply(convert_k_to_int)

# Tirando "," na coluna "commits"
data_refatorado['commits'] = data_refatorado['commits'].apply(remove_comma)

In [ ]:
# Trocando os nomes repetidos por um "novo" nome, impedindo que haja nomes repetidos no dataset

nomes = []
nomes_repitidos = {}

for i, name in enumerate(data_refatorado['name']):
    name = name.lower()
    if name not in nomes:
        nomes.append(name)
        nomes_repitidos[name] = 1
    else:
        nomes_repitidos[name] = nomes_repitidos[name] + 1
        nomes.append(f"{name}_{nomes_repitidos[name]}")
        data_refatorado.at[i, 'name'] = f"{name}_{nomes_repitidos[name]}"

In [ ]:
# Para os repositórios com topic_tag vazio, adicionamos o próprio tópico a lista

for i, value in enumerate(data_refatorado["topic_tag"]):
    if value == "[]":
        data_refatorado.at[i, "topic_tag"] = "['" + data_refatorado.at[i,"topic"] + "']"

In [ ]:
# Topicos escritos similarmente se tornam iguais

dicio={} # {letra: [topicos que começam com a letra]}
contador = 0
for index, row in data_refatorado.iterrows():
    cell_value = row["topic_tag"].lower() # valor da célula em str
    list_result = ast.literal_eval(cell_value) # valor da célula em list
    
    new_topic_tag = []
    for topico in list_result:
        estado = True
        first_letter = topico[0]
        if first_letter not in dicio.keys():
            dicio[first_letter] = [topico]
        else:
            for topico_dicio in dicio[first_letter]:
                if SequenceMatcher(None, topico, topico_dicio).ratio() >= 0.95:
                    topico = topico_dicio
                    estado = False
            if estado == True:
                dicio[first_letter].append(topico)
        new_topic_tag.append(topico)
                
    data_refatorado.at[index, "topic_tag"] = str(new_topic_tag)
    
    contador+=1
    print(f"{contador}/{len(data_refatorado)}", end = "\r")

## Mostrando o dataset refatorado

In [ ]:
data_refatorado

In [ ]:
# Mostrando apenas as colunas que utilizamos na análise
data_refatorado.loc[:, ['topic', 'name', 'star', 'topic_tag', 'commits']]

## Salvando o dataset refatorado

In [ ]:
data_refatorado.to_csv('Github_data_refatorado.csv', index=False)